# **Insper - Eletromagnetismo e Ondulatória** #


**Modelagem Projeto Eletromag. (Parte 2 do Projeto)**

In [217]:
#Vamos importar os módulos que precisamos
from cmath import *
from numpy import linalg
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import *

In [218]:
R1 = 1
R2 = 1
L1 = 10e-3
L2 = L1
C1 = 100e-6
C2 = C1
K = 0.2
M = (sqrt(L1*L2))*K

In [219]:
f = 60*10**3
W = 2*pi*f
Ufonte = 110
XL1 = 1j * W * L1
XL2 = 1j * W * L2
XM = 1j * W * M
XC1 = 1/(1j * W * C1)
XC2 = 1/(1j * W * C2)

*Modelo Paralelo*

In [220]:

def CalcularParalelo(Uf, Rc):
    Z=np.array([[R1+XL1+XC1, -XM],[-XM, XL2+R2+(Rc*XC2/(Rc+XC2))]])
    V=np.array([Uf,0])
    i=np.dot(linalg.inv(Z),V)
    return i[0], i[1]

In [221]:
Rcarga = 6
i1, i2 = CalcularParalelo(Ufonte, Rcarga)

print('No modelo em paralelo, com resistor na saída de %.2f Ω:' %Rcarga)
print('i1 (eficaz) = %.2f A' %np.abs(i1))
print('i2 (eficaz) = %.2f A' %np.abs(i2))
print('V1 (eficaz): %.2f V' %Ufonte)
print('V2 (eficaz): %.2f V' %(np.abs(i2*Rcarga)))

print('Relação N1/N2: %.2f' %(np.abs(sqrt(L1/L2))))
print('Relação i2/i1: %.2f' %(np.abs(i2)/np.abs(i1)))
print('Relação V1/V2: %.2f' %(np.abs(Ufonte)/np.abs(i2*Rcarga)))





No modelo em paralelo, com resistor na saída de 6.00 Ω:
i1 (eficaz) = 0.03 A
i2 (eficaz) = 0.01 A
V1 (eficaz): 110.00 V
V2 (eficaz): 0.04 V
Relação N1/N2: 1.00
Relação i2/i1: 0.20
Relação V1/V2: 3015.88


*Modelo em Série*

In [222]:
def CalcularEmSerie(Uf, Rc):
    Z=np.array([[R1+XL1+XC1, -XM],[-XM, XL2+R2+XC2+Rc]])
    V=np.array([Uf,0])
    i=np.dot(linalg.inv(Z),V)
    return i[0], i[1]

In [223]:
Rcarga = 5
i1, i2 = CalcularEmSerie(Ufonte, Rcarga)

print('No modelo em série, com resistor na saída de %.2f Ω:' %Rcarga)
print('i1 (eficaz) = %.2f A' %np.abs(i1))
print('i2 (eficaz) = %.2f A' %np.abs(i2))
print('V1 (eficaz): %.2f V' %Ufonte)
print('V2 (eficaz): %.2f V' %(np.abs(i2*Rcarga)))

print('Relação N1/N2: %.2f' %(np.abs(sqrt(L1/L2))))
print('Relação i2/i1: %.2f' %(np.abs(i2)/np.abs(i1)))
print('Relação V1/V2: %.2f' %(np.abs(Ufonte)/np.abs(i2*Rcarga)))





No modelo em série, com resistor na saída de 5.00 Ω:
i1 (eficaz) = 0.03 A
i2 (eficaz) = 0.01 A
V1 (eficaz): 110.00 V
V2 (eficaz): 0.03 V
Relação N1/N2: 1.00
Relação i2/i1: 0.20
Relação V1/V2: 3619.07


**Simulação do Circuito (Parte 3 do Projeto)**

In [224]:
def calcular_frequencia_ressonancia(L, C):
    return 1 / (2 * pi * np.sqrt(L * C))

fR = calcular_frequencia_ressonancia(L1, C1)
print('Frequência de ressonância do primário: %.2f Hz' %fR)

Frequência de ressonância do primário: 159.15 Hz


In [225]:
Usaida = i2 * Rcarga

def calcular_eficiencia(V1, I1, V2, I2):
    i2 = I2
    i1 = I1
    P_saida = np.real(0.5 * (V2/sqrt(2)) * i2.conjugate())
    P_entrada = np.real(0.5 * (V1/sqrt(2)) * i1.conjugate())
    eficiencia = P_saida / P_entrada

    return eficiencia

Ef = calcular_eficiencia(Ufonte, i1, Usaida, i2)
print('Eficiência do sistema: %.4f' %Ef)

Eficiência do sistema: 0.1613


In [226]:
def calcular_resistencia_pelicular(R_DC, f):
    return R_DC + (R_DC / (100 * 1e3))*f

R_DC = 0.1

Rp = calcular_resistencia_pelicular(R_DC, f)
print('Resistência pelicular: %.3f Ω' %Rp)

Resistência pelicular: 0.160 Ω


In [231]:
# Simulação para diferentes valores de C, em paralelo

C_values = [1e-7, 1.5e-7, 4.7e-7, 1e-6, 4.7e-6]


def simular_eficiencia(L, C_values, R_DC, V1, I1, V2, I2):
    plt.figure(figsize=(10, 6))

    for C in C_values:
        eficiencia = []
        tensao_saida = []
        freq = [calcular_frequencia_ressonancia(L, C)]
        freq_range = list(np.linspace(0, freq, 100))

        for f in freq_range:
            R_f = calcular_resistencia_pelicular(R_DC, f)
            
            # Redefinindo os parâmetros do circuito
            W = 2 * pi * f
            XL1 = 1j * W * L1
            XL2 = 1j * W * L2
            XM = 1j * W * M
            XC1 = 1/(1j * W * C1)
            XC2 = 1/(1j * W * C2)

            def CalcularParalelo(Uf, Rc):
                Z=np.array([[R1+XL1+XC1, -XM],[-XM, XL2+R2+(Rc*XC2/(Rc+XC2))]])
                V=np.array([Uf,0])
                i=np.dot(linalg.inv(Z),V)
                return i[0], i[1]

            I1_f, I2_f = CalcularParalelo(Ufonte, R_f)
            V2_f = I2_f * Rcarga
            
            eficiencia_f = calcular_eficiencia(V1, I1, V2_f, I2_f)
            eficiencia.append(eficiencia_f * 100)
            tensao_saida.append(V2_f)

        plt.plot([freq], eficiencia, label=f'C={C:.2e} F')
        print(eficiencia)
    
    plt.title('Eficiência do WPT com diferentes valores de capacitância')
    plt.xlabel('Frequência de ressonância (Hz)')
    plt.ylabel('Eficiência (%)')
    plt.legend()
    plt.grid(True)
    plt.show()

simular_eficiencia(L1, C_values, R_DC, Ufonte, i1, Usaida, i2)

C:\Users\aerac\AppData\Local\Temp\ipykernel_19100\14539870.py:23: RuntimeWarning: divide by zero encountered in divide
  XC1 = 1/(1j * W * C1)
C:\Users\aerac\AppData\Local\Temp\ipykernel_19100\14539870.py:23: RuntimeWarning: invalid value encountered in divide
  XC1 = 1/(1j * W * C1)
C:\Users\aerac\AppData\Local\Temp\ipykernel_19100\14539870.py:24: RuntimeWarning: divide by zero encountered in divide
  XC2 = 1/(1j * W * C2)
C:\Users\aerac\AppData\Local\Temp\ipykernel_19100\14539870.py:24: RuntimeWarning: invalid value encountered in divide
  XC2 = 1/(1j * W * C2)
C:\Users\aerac\AppData\Local\Temp\ipykernel_19100\14539870.py:27: RuntimeWarning: invalid value encountered in multiply
  Z=np.array([[R1+XL1+XC1, -XM],[-XM, XL2+R2+(Rc*XC2/(Rc+XC2))]])
C:\Users\aerac\AppData\Local\Temp\ipykernel_19100\14539870.py:27: RuntimeWarning: invalid value encountered in divide
  Z=np.array([[R1+XL1+XC1, -XM],[-XM, XL2+R2+(Rc*XC2/(Rc+XC2))]])


LinAlgError: Last 2 dimensions of the array must be square

<Figure size 1000x600 with 0 Axes>